## NLP Class Final Project Read Data

In [1]:
import pandas as pd
import re

In [2]:
pd.set_option('display.max_rows', 100)
pd.set_option('display.max_columns', None)
#pd.set_option('display.max_colwidth', None)

In [3]:
%%time

df_news_final_project = pd.read_parquet('https://storage.googleapis.com/msca-bdp-data-open/news_final_project/news_final_project.parquet', engine='pyarrow')
df_news_final_project.shape

CPU times: user 8.57 s, sys: 8.5 s, total: 17.1 s
Wall time: 22.4 s


(200332, 5)

In [4]:
df_news_final_project

,url,date,language,title,text
0,http://en.people.cn/n3/2021/0318/c90000-983012...,2021-03-18,en,Artificial intelligence improves parking effic...,\n\nArtificial intelligence improves parking e...
1,http://newsparliament.com/2020/02/27/children-...,2020-02-27,en,Children With Autism Saw Their Learning and So...,\nChildren With Autism Saw Their Learning and ...
2,http://www.dataweek.co.za/12835r,2021-03-26,en,"Forget ML, AI and Industry 4.0 – obsolescence ...","\n\nForget ML, AI and Industry 4.0 – obsolesce..."
3,http://www.homeoffice.consumerelectronicsnet.c...,2021-03-10,en,Strategy Analytics: 71% of Smartphones Sold Gl...,\n\nStrategy Analytics: 71% of Smartphones Sol...
4,http://www.itbusinessnet.com/2020/10/olympus-t...,2020-10-20,en,Olympus to Support Endoscopic AI Diagnosis Edu...,\n\nOlympus to Support Endoscopic AI Diagnosis...
...,...,...,...,...,...
200327,https://www.zazoom.it/2020-12-14/comviva-and-o...,2020-12-14,en,Comviva and Ooredoo Kuwait form strategic part...,\n\nComviva and Ooredoo Kuwait form strategic ...
200328,https://www.zdnet.com/article/afp-used-clearvi...,2020-04-15,en,AFP used Clearview AI facial recognition softw...,AFP used Clearview AI facial recognition softw...
200329,https://www.zdnet.com/article/from-data-to-kno...,2020-12-22,en,From data to knowledge and AI via graphs: Tech...,\n\nFrom data to knowledge and AI via graphs: ...
200330,https://yourstory.com/2020/10/ys-learn-helping...,2020-10-07,en,[YS Learn] From helping diabetic patients to m...,[YS Learn] From helping diabetic patients to m...


In [5]:
df_news_final_project = df_news_final_project[df_news_final_project['language'] == 'en']

### Text cleaning

In [6]:
%%time

df_news_final_project['cleaned_text'] = df_news_final_project['text'].apply(lambda x: re.sub(r'(?:\@|http?\://|https?\://|www)\S+','',str(x)))
df_news_final_project['cleaned_text'] = df_news_final_project['cleaned_text'].apply(lambda x: re.sub(r'(?:\n|\r|\t)','', str(x)))
df_news_final_project['cleaned_text'] = df_news_final_project['cleaned_text'].map(lambda x: re.sub(r'\B#\w*[a-zA-Z]+\w*','',str(x)))
df_news_final_project['cleaned_text'] = df_news_final_project['cleaned_text'].map(lambda x: re.sub('[^a-zA-Z0-9 @ . , : - _]', '', str(x)))

CPU times: user 2min 4s, sys: 3.79 s, total: 2min 8s
Wall time: 2min 8s


In [7]:
df_news_final_project[['text','cleaned_text']]

,text,cleaned_text
0,\n\nArtificial intelligence improves parking e...,Artificial intelligence improves parking effic...
1,\nChildren With Autism Saw Their Learning and ...,Children With Autism Saw Their Learning and So...
2,"\n\nForget ML, AI and Industry 4.0 – obsolesce...","Forget ML, AI and Industry 4.0 obsolescence s..."
3,\n\nStrategy Analytics: 71% of Smartphones Sol...,Strategy Analytics: 71 of Smartphones Sold Glo...
4,\n\nOlympus to Support Endoscopic AI Diagnosis...,Olympus to Support Endoscopic AI Diagnosis Edu...
...,...,...
200327,\n\nComviva and Ooredoo Kuwait form strategic ...,Comviva and Ooredoo Kuwait form strategic part...
200328,AFP used Clearview AI facial recognition softw...,AFP used Clearview AI facial recognition softw...
200329,\n\nFrom data to knowledge and AI via graphs: ...,From data to knowledge and AI via graphs: Tech...
200330,[YS Learn] From helping diabetic patients to m...,YS Learn From helping diabetic patients to man...


### Filtering relevant articles

In [8]:
relevant_words = ['artificial intelligence', 'data science', 'machine learning', 'generative AI', 'conversational AI','computer vision', 'natural language processing', 'deep learning', 'fraud detection', 'neural network', 'text analytics']

In [9]:
news_df = df_news_final_project[df_news_final_project['cleaned_text'].str.contains('|'.join(relevant_words))]

In [10]:
news_df.shape

(126817, 6)

In [11]:
import sys
import os
import numpy as np
import pandas as pd
import tensorflow as tf
import re
import string

import nltk as nltk
from nltk.corpus import stopwords
from nltk.stem.wordnet import WordNetLemmatizer

print('Python Version: ' + sys.version)
print('TensorFlow Version: ' + tf.__version__)

2023-05-17 01:30:58.025264: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-05-17 01:30:58.203288: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2023-05-17 01:30:58.203323: I tensorflow/compiler/xla/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.
2023-05-17 01:30:59.156215: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory
2023-

Python Version: 3.7.12 | packaged by conda-forge | (default, Oct 26 2021, 06:08:21) 
[GCC 9.4.0]
TensorFlow Version: 2.11.0


In [12]:
news_df.head()

,url,date,language,title,text,cleaned_text
0,http://en.people.cn/n3/2021/0318/c90000-983012...,2021-03-18,en,Artificial intelligence improves parking effic...,\n\nArtificial intelligence improves parking e...,Artificial intelligence improves parking effic...
3,http://www.homeoffice.consumerelectronicsnet.c...,2021-03-10,en,Strategy Analytics: 71% of Smartphones Sold Gl...,\n\nStrategy Analytics: 71% of Smartphones Sol...,Strategy Analytics: 71 of Smartphones Sold Glo...
13,https://9to5mac.com/2021/03/25/apple-bought-mo...,2021-03-26,en,Apple bought more AI companies than anyone els...,\nApple bought more AI companies than anyone e...,Apple bought more AI companies than anyone els...
14,https://abcnews.go.com/Nightline/video/trump-d...,2023-04-06,en,Video Trump deepfakes on social media prompt w...,\n\nVideo Trump deepfakes on social media prom...,Video Trump deepfakes on social media prompt w...
15,https://abcnews.go.com/Technology/video/eu-imp...,2020-02-19,en,EU imposes sweeping regulations on facial reco...,\n\nEU imposes sweeping regulations on facial ...,EU imposes sweeping regulations on facial reco...


In [13]:
import time
import math
import re
from pprint import pprint
from textblob import TextBlob
import pandas as pd
import numpy as np

import matplotlib.pyplot as plt


import nltk as nltk
from nltk.corpus import stopwords
# from nltk.stem.wordnet import WordNetLemmatizer
import spacy
import multiprocessing
import string


import gensim
from gensim import corpora, models
from gensim.models.ldamulticore import LdaMulticore
from gensim.utils import simple_preprocess
from gensim.models import CoherenceModel

# import pyLDAvis.gensim
import pyLDAvis
import pyLDAvis.gensim_models as gensimvis
pyLDAvis.enable_notebook()

2023-05-17 01:31:00.825441: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcuda.so.1'; dlerror: libcuda.so.1: cannot open shared object file: No such file or directory
2023-05-17 01:31:00.825488: W tensorflow/compiler/xla/stream_executor/cuda/cuda_driver.cc:265] failed call to cuInit: UNKNOWN ERROR (303)
2023-05-17 01:31:00.825514: I tensorflow/compiler/xla/stream_executor/cuda/cuda_diagnostics.cc:156] kernel driver does not appear to be running on this host (nlp): /proc/driver/nvidia/version does not exist


In [14]:
#nltk.download('wordnet')

In [15]:
%%time

from multiprocessing import Pool

stop = set(stopwords.words('english'))
exclude = set(string.punctuation)
lemma = WordNetLemmatizer()
def clean(doc):
    stop_free = " ".join([i for i in doc.lower().split() if i not in stop])
    punc_free = ''.join(ch for ch in stop_free if ch not in exclude)
    normalized = " ".join(lemma.lemmatize(word) for word in punc_free.split())
    return normalized

def parallel_clean(data):
    with Pool() as pool:
        results = pool.map(clean, data)
    return [result.split() for result in results]

news_list = news_df['cleaned_text'].tolist()
news_clean = parallel_clean(news_list)

title_list = news_df['title'].tolist()
title_clean = parallel_clean(title_list)

CPU times: user 13.3 s, sys: 10.7 s, total: 24 s
Wall time: 1min 34s


In [16]:
%%time

# Creating the term dictionary of our courpus, where every unique term is assigned an index. 
dictionary = corpora.Dictionary(title_clean)

# Converting list of documents (corpus) into Document Term Matrix using dictionary prepared above.
doc_term_matrix = [dictionary.doc2bow(doc) for doc in news_clean]

CPU times: user 1min 12s, sys: 3.86 s, total: 1min 16s
Wall time: 1min 16s


In [17]:
num_processors = multiprocessing.cpu_count()
num_processors

16

### Performing LDA on the corpus for 10 topics

In [18]:
%%time

num_topics = 10
passes = 10
workers = num_processors-1
eval_every = None

ldamodel = LdaMulticore(corpus=doc_term_matrix,
                       id2word=dictionary,
                       eta='auto',
                       num_topics=num_topics,
                       passes=passes,
                       eval_every=eval_every,
                       workers = workers)

CPU times: user 6min 29s, sys: 23.4 s, total: 6min 53s
Wall time: 18min 2s


In [19]:
print(*ldamodel.print_topics(num_topics=num_topics, num_words=10), sep='\n\n')

(0, '0.010*"ai" + 0.008*"new" + 0.005*"like" + 0.004*"republic" + 0.004*"say" + 0.004*"people" + 0.004*"one" + 0.004*"image" + 0.004*"human" + 0.004*"time"')

(1, '0.012*"patient" + 0.012*"health" + 0.011*"ai" + 0.008*"medical" + 0.007*"clinical" + 0.006*"care" + 0.006*"healthcare" + 0.005*"technology" + 0.005*"cancer" + 0.005*"news"')

(2, '0.015*"ai" + 0.011*"news" + 0.009*"data" + 0.009*"technology" + 0.009*"business" + 0.007*"service" + 0.007*"product" + 0.006*"industry" + 0.006*"security" + 0.005*"best"')

(3, '0.011*"ai" + 0.009*"news" + 0.007*"company" + 0.007*"technology" + 0.006*"new" + 0.006*"u" + 0.005*"business" + 0.004*"weather" + 0.004*"valley" + 0.004*"medium"')

(4, '0.021*"ai" + 0.008*"intelligence" + 0.007*"artificial" + 0.007*"technology" + 0.006*"u" + 0.005*"new" + 0.005*"said" + 0.005*"use" + 0.005*"system" + 0.004*"company"')

(5, '0.026*"data" + 0.013*"learning" + 0.010*"science" + 0.010*"machine" + 0.008*"ai" + 0.008*"model" + 0.005*"new" + 0.005*"student" + 0.0

- Topic 0: "AI, New Republic, People, Human: Exploring the Impact of AI on Society"
- Topic 1: "Patient Health, Medical Care, AI: Revolutionizing Healthcare with AI Technology"
- Topic 2: "AI, News, Data, Technology: Unveiling the Latest Trends in AI and Data-driven Business"
- Topic 3: "AI, News, Technology, Business: Insights into the Ever-evolving AI Landscape"
- Topic 4: "AI, Artificial Intelligence, Technology: Harnessing the Power of AI for Innovation"
- Topic 5: "Data Science, Machine Learning, AI: Advancements in Data-driven Learning and AI Models"
- Topic 6: "AI, Company, Customer Experience: Transforming Businesses with AI-driven Solutions"
- Topic 7: "ChatGPT, AI, Google: Recent Developments and News in AI, including ChatGPT"
- Topic 8: "AI, Gray, Data, Platform: Leveraging AI and Data for Effective Solutions"
- Topic 9: "AI Market, Artificial Intelligence Report: Analyzing the Growth and Trends in the Global AI Industry"

In [20]:
%%time

lda_display = gensimvis.prepare(ldamodel, doc_term_matrix, dictionary, sort_topics=False, mds='mmds')
pyLDAvis.display(lda_display)

/opt/conda/lib/python3.7/site-packages/pyLDAvis/_prepare.py:247: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
  by='saliency', ascending=False).head(R).drop('saliency', 1)


CPU times: user 16min 18s, sys: 17min 7s, total: 33min 26s
Wall time: 3min 38s


In [21]:
topic_distribution = ldamodel.get_document_topics(doc_term_matrix)

In [22]:
dominant_topics = [max(topic, key=lambda x: x[1])[0] for topic in topic_distribution]

In [23]:
news_df['topic'] = dominant_topics

/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [24]:
news_df

,url,date,language,title,text,cleaned_text,topic
0,http://en.people.cn/n3/2021/0318/c90000-983012...,2021-03-18,en,Artificial intelligence improves parking effic...,\n\nArtificial intelligence improves parking e...,Artificial intelligence improves parking effic...,4
3,http://www.homeoffice.consumerelectronicsnet.c...,2021-03-10,en,Strategy Analytics: 71% of Smartphones Sold Gl...,\n\nStrategy Analytics: 71% of Smartphones Sol...,Strategy Analytics: 71 of Smartphones Sold Glo...,8
13,https://9to5mac.com/2021/03/25/apple-bought-mo...,2021-03-26,en,Apple bought more AI companies than anyone els...,\nApple bought more AI companies than anyone e...,Apple bought more AI companies than anyone els...,0
14,https://abcnews.go.com/Nightline/video/trump-d...,2023-04-06,en,Video Trump deepfakes on social media prompt w...,\n\nVideo Trump deepfakes on social media prom...,Video Trump deepfakes on social media prompt w...,7
15,https://abcnews.go.com/Technology/video/eu-imp...,2020-02-19,en,EU imposes sweeping regulations on facial reco...,\n\nEU imposes sweeping regulations on facial ...,EU imposes sweeping regulations on facial reco...,4
...,...,...,...,...,...,...,...
200324,https://www.wymt.com/prnewswire/2023/03/01/dri...,2023-03-01,en,"Drift Introduces GPT Integration, Advancing AI...","Drift Introduces GPT Integration, Advancing AI...","Drift Introduces GPT Integration, Advancing AI...",6
200325,https://www.wymt.com/prnewswire/2023/04/28/uro...,2023-04-28,en,Urologists Explore AI for Prostate Cancer Dete...,Urologists Explore AI for Prostate Cancer Dete...,Urologists Explore AI for Prostate Cancer Dete...,1
200326,https://www.yorkdispatch.com/story/opinion/con...,2023-04-16,en,Does AI mean the 4-day workweek is almost here?,Does AI mean the 4-day workweek is almost here...,Does AI mean the 4day workweek is almost hereT...,4
200329,https://www.zdnet.com/article/from-data-to-kno...,2020-12-22,en,From data to knowledge and AI via graphs: Tech...,\n\nFrom data to knowledge and AI via graphs: ...,From data to knowledge and AI via graphs: Tech...,5


### Saving the new filtered data as a parquet file

In [25]:
news_df.to_parquet("news_topics.parquet")